<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Комментарий ревьюера</b>


Привет, Максим! Давай знакомиться! Меня зовут Дмитрий Махортов, и я буду проверять твой проект. Сразу предлагаю общение на "ты" 🙂, но если тебе это не комфортно, то дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе погрузиться в увлекательный мир работы с данными и вырасти в крепкого профи. Это отдаленная цель. А ближайшая - сделать твою работу еще лучше )).


Вижу, что ты очень старался, делая проект. Все ключевые этапы в работе выполнены, и в целом с проектом ты справился. 
    
Расскажу как обычно проходит проверка:    
Бывают моменты, которые требуют пристального внимания. Комментарии по ним выделены <span style='background-color:#F7B3A4'> красным цветом </span> и обозначены значком 🛑. После их доработки проект будет принят.  🙂

<span style='background-color:#F9EDA6'>Жёлтым цветом </span> и значком ⚠️ выделено то, что в следующий раз можно сделать по-другому. Можно учесть эти комментарии при выполнении будущих заданий или доработать текущий проект (однако это не обязательно). 

<span style='background-color:#B7EBA7'> Зеленым цветом </span> и значком ✅ отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах. Или советы "со звездочкой", которые помогут в будущем.

    
Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом**. Выбери для своих комментариев какой-то заметный цвет, так мне будет легче отследить изменения. Пожалуйста, **не перемещай, не изменяй и не удаляй мои комментарии**. Всё это поможет выполнить повторную проверку твоего проекта оперативнее».
</div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Отлично, все нужные библиотеки импортированы в начале ноутбука.Это хорошая практика.</div>

In [44]:
df = pd.read_csv("/datasets/users_behavior.csv")

display(df.shape)
display(df.head(30)) 

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Не стоит применять print() к датафреймам, вывод теряет всю красоту. Используй функцию display(). А иногда можно вообще обойтись без вызова дополнительной фукнции, достаточно просто обратиться к объекту в последней строке ячейки.
</div>

Спасибо! Поменяю привычку<div class="alert alert-success">

Столбцы содержат следующую информацию: 
    
сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Посмотрим на типы столбцов

In [3]:
df.info()
print('')
print('Количество полных дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Количество полных дубликатов: 0


Посчитаем количество пропусков в наборе данных

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Данные загружены и исследованы.</div>

## Разбейте данные на выборки

Разделим датасет на обучающую и тестовую выборки

In [6]:
train, test = train_test_split(df, test_size=0.25, random_state=12345)

In [7]:
features_one = test.drop('is_ultra', axis=1)
target_one = test['is_ultra']

Создадим переменные для признаков и целевого признака

In [8]:
features_valid, features_test, target_valid, target_test = train_test_split(features_one, target_one, test_size=0.5, random_state=12345)

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Старайся не делать такие длинные строки. Стандарт PEP8 регулирует длину строки 79 символов. Придерживаться такого очень сложно, но лучше хотя-бы вмещать в экран, чтобы не приходилось использовать scroll.
</div>

In [9]:
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

In [10]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(2410, 4)
(2410,)
(402, 4)
(402,)


<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Чтобы не путать читателя, лучше вывести размерности всех трех выборок
</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Молодец, корректно разделил данные на три выборки.
На будущее, обрати внимание на аргумент stratify функции train_test_split. Стратификация особенно важна, когда у нас мало данных и возможен дисбаланс классов в обучении / валидации / тесте.
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Размеры тестовой и валидационных выборок меньше, чем рекомендованные 20%. Однако вполне допустимо разотать и с выборками такого размера.
</div>

## Исследуйте модели

Перебем гиперпараметры в дереве решений через цикл Tree Classifier 

In [11]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)

    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) 

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.763681592039801
max_depth = 2 : 0.7935323383084577
max_depth = 3 : 0.7985074626865671
max_depth = 4 : 0.7985074626865671
max_depth = 5 : 0.7985074626865671


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Дерево решений исследовано корректно:
    
 - исследовано 5 значений гиперпараметров
 - модель обучена на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Можно дополнительно выводить оценки качества для обучающего (train) набора. 
</div>

Подходящая max_depth = 3

Качество = 0.7985074626865671

Проверим качество с помощью логистической регрессии 

In [12]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)

Качество: 0.7039800995024875


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Логистическая регрессия исследована корректно.
</div>


Проверим качество с помощью Random Forest 

In [13]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=412)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est, "   минимальное количество объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 6     Количество деревьев: 20    минимальное количество объектов в узле: 3   Качество: 0.8109452736318408


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Случайный лес исследован корректно:
    
 - исследовано 9х10х3 комбинаций значений гиперпараметров
 - модель обучена на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле
</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Рассмотрены все три изученных алгоритма (решающее дерево, случайный лес, логистическая регрессия)
</div>


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Нужно сделать выводы, о том, какие модели работают хуже, а какие лучше. И выбрать одну лучшую модель, включая гиперпараметры.</div>

Проверены три модели: деревья решений, логистическая регрессия и случайный лес.

Самое высокое качество модели (0.810) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=20 и min_samples_leaf=3

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
<b>v2</b> 👍 </div>

## Проверьте модель на тестовой выборке

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
К данному этапу лучше подходить с одной выбранной моделью (ты это сделал). Чтобы избежать искушения вдруг "поменять" модель по  результатам тестирования. Ведь любое использование тестовой выборки для улучшения модели (в том числе выбор лучшей модели) - это нарушение.</div>

Проверим качество с помощью Tree Classifier

In [14]:
model = DecisionTreeClassifier(max_depth=8, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7960199004975125


Проверим качество с помощью логистической регрессии

In [15]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7039800995024875


Проверим качество с помощью Random Forest

In [16]:
model = RandomForestClassifier(max_depth=7, n_estimators=40, min_samples_leaf=3, random_state=228)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8084577114427861


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Молодец: 
    
  - выбрана лучшая модель
  - тестирование произведено корректно 
  - достигнута хорошая точность на тестовой выборке
</div>


<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Я вижу что ты обучаешь модель на только на обучающей выборке. Сейчас, когда гиперпараметры подобраны и валидационная выборка нам не нужна, мы можем обучить модель на большой выборке (обучающая + валидационная), а тестирование новой модели произвести уже на тестовой выборке.
    
Объединить выборки можно с помощью pd.concat
</div>

Самое высокое качество модели (0.808) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=40 и min_samples_leaf=3

## (бонус) Проверьте модели на адекватность

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
В этом задании предполагался простой подход. Нужно измерить точность для "наивного" алгоритма, например предсказывающего случайный ответ или константный. Рекомендую посмотреть, какую  точность дает "наивный" алгоритм, всегда предсказывающий самый популярный ответ (у нас это 0). Вычислить это можно вручную или используя DummyClassifier.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


## Вывод

Проверены три модели: деревья решений, логистическая регрессия и случайный лес.


Самое высокое качество модели (0.808) достигается при использовании Random Forest с гиперпараметрами max_depth=7 , n_estimators=40 и min_samples_leaf=3

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий</b>

В целом мне все понравилось: твоя работа выполнена на хорошем уровне с минимальными помарками.
Также работа выглядит аккуратной и хорошо оформленной.
Вижу, что тебе дается python, и инструментарий машинного обучения.

Что нужно доработать обязательно (красный комментарий) : после завершения исследования моделей с использованием валидационной выборки нужно написать выводы в которых выбрать одну лучшую модель (вместе с гиперпараметрами)
    
Желтые комментарии на твое усмотрение.
     
Жду твоих исправлений :)
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v2</b>

Проект принят. Поздравляю и желаю дальнейших успехов!</div>